# DQN для выбора лимитов портфеля с максимизацией Sharpe Ratio

In [ ]:
# Установка!pip install numpy matplotlib torch

In [ ]:
# Импортimport numpy as npimport torchimport torch.nn as nnimport torch.optim as optimimport randomimport matplotlib.pyplot as pltfrom collections import deque

In [ ]:
# Класс средыclass SharpeEnv:    def __init__(self):        self.state = np.array([0.1, 0.1, 0.1])  # crypto, stocks_ru, bonds_ru        self.step_size = 0.01        self.bounds = [0.0, 0.3]        self.action_space = 6    def reset(self):        self.state = np.random.uniform(0.0, 0.3, size=3)        return self.state    def step(self, action):        idx = action // 2        delta = self.step_size if action % 2 == 0 else -self.step_size        self.state[idx] = np.clip(self.state[idx] + delta, *self.bounds)        sharpe = -np.sum((self.state - 0.2)**2) + 1.0  # Заглушка Sharpe        reward = sharpe        return self.state.copy(), reward, False, {}

In [ ]:
# DQN модельclass DQN(nn.Module):    def __init__(self, state_dim, action_dim):        super(DQN, self).__init__()        self.net = nn.Sequential(            nn.Linear(state_dim, 64),            nn.ReLU(),            nn.Linear(64, action_dim)        )    def forward(self, x):        return self.net(x)

In [ ]:
# Основной цикл обученияenv = SharpeEnv()q_net = DQN(3, 6)target_net = DQN(3, 6)target_net.load_state_dict(q_net.state_dict())optimizer = optim.Adam(q_net.parameters(), lr=1e-3)buffer = deque(maxlen=10000)batch_size = 64gamma = 0.99epsilon = 0.2rewards_log = []for ep in range(300):    s = env.reset()    total_reward = 0    for _ in range(20):        if random.random() < epsilon:            a = random.randint(0, 5)        else:            with torch.no_grad():                a = q_net(torch.FloatTensor(s)).argmax().item()        s2, r, done, _ = env.step(a)        buffer.append((s, a, r, s2))        s = s2        total_reward += r        if len(buffer) >= batch_size:            batch = random.sample(buffer, batch_size)            states, actions, rewards, next_states = zip(*batch)            states = torch.FloatTensor(states)            actions = torch.LongTensor(actions)            rewards = torch.FloatTensor(rewards)            next_states = torch.FloatTensor(next_states)            q_vals = q_net(states).gather(1, actions.unsqueeze(1)).squeeze()            next_q = target_net(next_states).max(1)[0]            target = rewards + gamma * next_q            loss = nn.functional.smooth_l1_loss(q_vals, target.detach())            optimizer.zero_grad()            loss.backward()            optimizer.step()    if ep % 10 == 0:        target_net.load_state_dict(q_net.state_dict())    rewards_log.append(total_reward)plt.plot(rewards_log)plt.title("Sharpe (Reward) по эпизодам - DQN")plt.xlabel("Эпизод")plt.ylabel("Reward (Sharpe Proxy)")plt.grid(True)plt.show()